In [1]:
import dask
import time

import os

from dask_cuda import initialize, LocalCUDACluster
from dask.distributed import Client, default_client

In [2]:
import rmm
import cupy as cp

from rmm.allocators.cupy import rmm_cupy_allocator

def set_mem():
    rmm.reinitialize(managed_memory=True)
    cp.cuda.set_allocator(rmm_cupy_allocator)

In [3]:
preprocessing_gpus="0,1,2,3,4,5,6,7"
#preprocessing_gpus="0"

In [4]:
%%time
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=preprocessing_gpus)

client = Client(cluster)    

set_mem()
client.run(set_mem)

client

CPU times: user 527 ms, sys: 719 ms, total: 1.25 s
Wall time: 12.1 s


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 1.95 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8183,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 1.95 TiB
Comm: tcp://127.0.0.1:8467,Total threads: 1
Dashboard: http://127.0.0.1:8041/status,Memory: 250.00 GiB
Nanny: tcp://127.0.0.1:8299,


In [5]:
import rapids_singlecell as rsc
import anndata as ad
import scanpy as sc

In [6]:
from cupyx.scipy import sparse

In [7]:
dask.array.register_chunk_type(sparse.csr_matrix)

In [8]:
from anndata.experimental import read_elem_as_dask

from pathlib import Path
import os
import h5py

SPARSE_CHUNK_SIZE = 200_000
data_pth = "../2024_gpu_severin.dicks/notebooks/h5/6M_CxG.h5ad"


f = h5py.File(data_pth)
X = f["X"]
shape = X.attrs["shape"]
adata = ad.AnnData(
    X = read_elem_as_dask(X, (SPARSE_CHUNK_SIZE, shape[1])),
    obs = ad.read_elem(f["obs"]),
    var = ad.read_elem(f["var"]))
f.close()



/ictstr01/home/icb/severin.dicks/git/anndata/src/anndata/_core/anndata.py:1759: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [9]:
import time
t1 = time.time()

In [10]:
adata.X = adata.X.map_blocks(sparse.csr_matrix, meta=sparse.csr_matrix(cp.array((1.,))), dtype=cp.float32)


In [11]:
#rsc.pp.flag_gene_family(adata, gene_family_name="MT", gene_family_prefix="mt-")


In [12]:
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

dask.array<csr_matrix, shape=(6948864, 19331), dtype=float32, chunksize=(200000, 19331), chunktype=cupyx.csr_matrix>

In [13]:
t2 = time.time()

In [14]:
%%time
rsc.pp.calculate_qc_metrics(adata)

CPU times: user 735 ms, sys: 178 ms, total: 914 ms
Wall time: 10.2 s


In [15]:
adata = adata[(adata.obs["n_genes_by_counts"]<=10000) 
            & (adata.obs["n_genes_by_counts"]>=200)].copy()
rsc.pp.filter_genes(adata, min_count=10)

/ictstr01/home/icb/severin.dicks/git/anndata/src/anndata/_core/index.py:159: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return a[subset_idx]
/ictstr01/home/icb/severin.dicks/git/anndata/src/anndata/_core/anndata.py:1759: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


filtered out 132 genes based on n_cells_by_counts


In [16]:
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

/home/icb/severin.dicks/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 53.08 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


dask.array<getitem, shape=(6937990, 19199), dtype=float32, chunksize=(199767, 19199), chunktype=cupyx.csr_matrix>

In [17]:
%%time
rsc.pp.normalize_total(adata,target_sum = 10000)


CPU times: user 11.1 ms, sys: 3.65 ms, total: 14.8 ms
Wall time: 137 ms


In [18]:
from rapids_singlecell._compat import DaskArray, DaskClient, _get_dask_client

In [19]:
%%time
rsc.pp.log1p(adata)

CPU times: user 651 μs, sys: 848 μs, total: 1.5 ms
Wall time: 1.34 ms


In [20]:
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

dask.array<lambda, shape=(6937990, 19199), dtype=float32, chunksize=(199767, 19199), chunktype=cupyx.csr_matrix>

In [21]:
%%time
rsc.pp.highly_variable_genes(adata,n_top_genes=5000, flavor="cell_ranger")
adata = adata[:,adata.var.highly_variable].copy()

CPU times: user 369 ms, sys: 112 ms, total: 482 ms
Wall time: 4.75 s


/ictstr01/home/icb/severin.dicks/git/anndata/src/anndata/_core/index.py:159: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return a[subset_idx]
/ictstr01/home/icb/severin.dicks/git/anndata/src/anndata/_core/anndata.py:1759: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [22]:
adata.shape

(6937990, 5000)

In [23]:
n_rows = adata.shape[0]
n_cols = adata.shape[1]
cols_per_worker = int(n_rows / 8)
adata.X = adata.X.rechunk((cols_per_worker, n_cols)).persist()

adata.X.compute_chunk_sizes()

dask.array<rechunk-merge, shape=(6937990, 5000), dtype=float32, chunksize=(867248, 5000), chunktype=cupyx.csr_matrix>

In [24]:
%%time
rsc.pp.scale(adata, zero_center= False)
adata.X = adata.X.persist()
adata.X.compute_chunk_sizes()

/home/icb/severin.dicks/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 26.49 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 494 ms, sys: 96.4 ms, total: 591 ms
Wall time: 4.97 s


dask.array<scale_kernel_csr, shape=(6937990, 5000), dtype=float32, chunksize=(867248, 5000), chunktype=cupyx.csr_matrix>

In [25]:
%%time
rsc.pp.pca(adata, n_comps = 100,mask_var=None)
adata.obsm["X_pca"]=adata.obsm["X_pca"].persist()
adata.obsm["X_pca"].compute_chunk_sizes()

CPU times: user 1.36 s, sys: 349 ms, total: 1.71 s
Wall time: 17.3 s


dask.array<_transform, shape=(6937990, 100), dtype=float32, chunksize=(867248, 100), chunktype=cupy.ndarray>

In [26]:
print((time.time()-t2))
print((time.time()-t1)/60)

60.341403007507324
6.557059117158254
